# 1. Word2Vec 모델 학습을 위한 전처리


## 1) Tag 데이터 로드

먼저 태그데이터를 읽어옵니다. <br>
pandas를 사용하여 읽어온 데이터를 데이터프레임 형태로 변환합니다. <br>
데이터는 MovieLens의 데이터셋을 사용하였습니다. <br>


In [1]:
import pandas as pd

tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)
tag_df.head(10)

# 유저별 영화별 태그 데이터입니다.


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195
5,14,318,justice,1442615192
6,14,480,Dinosaurs,1443148563
7,14,593,psychothriller,1444014286
8,14,1682,philosophy,1442615158
9,14,1682,surveillance,1442615167


## 2) 유저별 태그데이터 배열 생성
로드한 데이터를 잘라서 유저별 영화별 배열을 생성합니다. <br>

<br>

#### 예 : <br>
#### [유저1-영화1-태그1  ,  유저1-영화1-태그2  ,  유저1-영화1-태그3 , ...] <br>
가장 먼저 태그들이 정렬됩니다. 이때, 태그는 시간의 순서에 따라 정렬합니다. <br>
비슷한시간대에 비슷한 태그를 작성한다는 가정하에 정렬하였습니다. <br>

<br>

#### [유저1-영화2-태그1  ,  유저1-영화2-태그2  ,  유저1-영화2-태그3 , ...] <br>
영화가 바뀌면 새로운 어레이가 생성됩니다. <br>
유저1이 영화1번, 영화2번에 작성한 태그는 각각 다른 어레이에 들어갑니다. <br>

<br>

#### [유저2-영화1-태그1  ,  유저2-영화1-태그2  ,  유저2-영화1-태그3 , ...] <br>
유저 1이 작성한 태그가 끝나면 유저2의 태그가 시작됩니다. <br>
마찬가지로 같은영화에 작성한 태그는 같은 어레이에 들어가고 <br>
다른영화에 작성한 태그는 다른 어레이에 들어갑니다. <br>
유저2가 끝나면 유저n 까지 반복합니다. <br> 


In [2]:
from src.word2vec_mylib import tag_manager as tm

tag_array = tm.get_tag_by_user(tag_df, start_movie=110, start_user=14)

for i in range(0, 10):
    print(tag_array[i])
    
# 위의 방식대로 처리한 태그데이터가 담긴 배열입니다.

['medieval', 'epic']
['sci-fi', 'space action']
['justice', 'imdb top 250']
['dinosaurs']
['psychothriller']
['philosophy', 'surveillance']
['epic']
['pixar']
['dinosaurs']
['sci-fi', 'classic sci-fi', 'harrison ford', 'must see']


# 2. Word2Vec 모델 학습


## 1) 모델 학습
<br>
위처럼 생성한 배열을 Word2Vec 모델에 넣어 학습시킵니다. <br>
Word2Vec 모델은 Gensim의 라이브러리를 사용하였습니다. <br> 
<br>
Skip gram 방식을 사용하였고, Window Size는 256 차원입니다. <br>
100번 이하로 나온 태그들은 제거하고, 100번 학습을 시켰습니다. <br>


In [3]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(tag_array,
                 size=256,
                 window=3,
                 workers=8,
                 min_count=100,
                 iter=100,
                 sg=1)

model.init_sims(replace=True)


C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2) 모델 테스트

<br>

모델 학습이 잘 되었는지 테스트를 해봅니다. <br>
marvel 태그와 비슷한 태그를 출력합니다. <br>

In [5]:
print("marvel 태그와 유사한 태그 목록 :  \n")
model.wv.most_similar("marvel")


marvel 태그와 유사한 태그 목록 :  



C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('marvel cinematic universe', 0.5684163570404053),
 ('mcu', 0.543043851852417),
 ('superhero', 0.5212122201919556),
 ('the avengers', 0.5133495926856995),
 ('thor', 0.5008189678192139),
 ('iron man', 0.48622554540634155),
 ('x-men', 0.47844862937927246),
 ('comic book', 0.47578462958335876),
 ('captain america', 0.4704328775405884),
 ('superheroes', 0.39934277534484863)]

# 3. 추천 섹션을 위한 전처리


## 1) 데이터로드

추천섹션에 필요한 모든 데이터를 로드하고 전처리를 거칩니다 <br>
태그데이터는 태그와 영화ID 외에 필요없는 데이터를 모두 잘라내며, <br>
평점데이터에서 3점아래의 영화는 추천하기 어렵다고 판단되어 <br>
3점 아래의 영화를 모두 잘라냅니다. <br>
<br>
그리고 필요한 모든 파이썬 코드를 import 합니다. <br>
로직구현에 필요한 코드가 너무 복잡하고 길어서 mylib 패키지로 모두 옮겼습니다 <br>
세부로직을 보려면 mylib 패키지의 코드를 참고하세요.

In [9]:
from src.word2vec_mylib import movie_manager as mm
from src.word2vec_mylib import recommend_manager as rm
from src.word2vec_mylib import tag_manager as tm
import math

tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)
tag_df = mm.cut_userId_timestamp(tag_df)

rating = pd.read_csv('./data/ratings_large.csv')
rating_df = pd.DataFrame(rating)
rating_df = mm.get_higher_rating_movie(rating_df)

movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)

# 4. 추천섹션

## 1) 영화 추천 섹션 - 가중치와 유효태그

<br>

### (1) 사용자가 입력한 태그라고 가정하고 영화번호를 입력합니다<br><br>

### (2) 위 영화의 모든 태그를 빈도수 순으로 뽑아냅니다<br><br>

### (3) 본 논문에서 (해당 태그가 달린횟수 / 모든태그가 달린 횟수) 를 반올림한 값을 "기중치"라고 명명합니다.
### 이 가중치 값이 0.1 이상인 영화를 "유효태그"라고 명명하고, 이 유효태그 목록을 생성합니다<br><br>
예를들어, 영화에 태그가 달린 횟수가 전체 100회일때, 만약 a라는 태그가 15번 나왔다면, a는 0.2 {0.15 => 반올림 0.2} 가중치를 가집니다.<br><br>
또 예를들어, 영화에 태그가 달린 횟수가 15회인데, 모든 태그가 1번씩 나왔다면 모든태그는 0.1 {0.0667 => 반올림 0.1}의 가중치를 가집니다.<br>




## 2) 영화 추천 섹션 - 유사태그 선별

<br>

### (4) 각 "유효태그"의 가중치에 비례해 Word2Vec모델에서 "유사태그"를 선출하고 "유효태그 + 유사태그" 목록을 생성합니다.<br>
### 모든 태그들의 가중치중 가장 낮은 가중치값을 갖는 태그를 제외한 태그들을 모델에 넣어서 가중치*10 개 만큼의 "유사태그"를 선출합니다 <br><br>
(예1. 태그n=0.2, 태그m=0.1, 나머지=0...등)<br><br>
이 영화에서 가장 낮은 가중치는 0이기 때문에, 0을 제외한 n(0.2) , m(0.1) 등의 가중치를 갖는 태그들의 유사태그를 선출합니다. <br><br>
태그n을 모델에 넣어 가장 유사한 태그2개(0.2*10)를 선별, 태그m을 모델에 넣어 가장 유산한태그1개(0.1*10)를 선별합니다<br><br>
그리고 이 유사태그리스트[n1, n2, m1]와 유효태그(0.1이상)리스트[n, m]을 합친 목록을 생성합니다. ==> [n, n1, n2, m, m1] <br><br><br>

(예2. 모든태그가 0.1인경우)<br><br>
이 영화에서 가장 낮은 가중치는 0.1이기 때문에, 모든 태그의 유사태그를 선별하지 않습니다. <br><br>
어떤 태그가 중요한 태그인지 모르는 상황에서, 그 태그들의 유사태그를 선별하게 되면, 그태그 리스트들이 편향되어, <br><br>
영화를 대표할수 없는 태그 리스트가 될 가능성이 높습니다. <br><br>
그러나 모든태그가 0.1이상의 가중치를 갖기에 모든 태그가 "유효태그"가 되어 <br><br>
영화의 모든 유효태그가 곧 "유사태그 + 유효태그"리스트가 됩니다.<br><br><br>


(예3. 태그가 2개있는데 두 태그가 0.5의 가중치를 갖는 경우)<br><br>
위와 마찬가지로 가장 낮은 가중치는 0.5이기때문에 이 영화에서도 유사태그를 선별하지 않습니다. <br><br>
그러나 태그의 수가 너무 적기 때문에 이후에 "태그리스트" 조정과정에서 추가로 태그를 선별하게 됩니다.<br><br>
일단 지금 과정에서는 그 태그 2개만이 "유사태그 + 유효태그" 리스트가 됩니다. <br>







## 3) 영화 추천 섹션 - 태그리스트 조정과정

모든 영화의 "유사태그 + 유효태그" 갯수가 비슷해야 비슷한 수의 영화를 추천후보영화로 선정할 수 있습니다. <br><br>
때문에 모든 영화들의 태그를 비슷한 수치로 고정시킵니다. <br>
든 영화의 "유사태그 + 유효태그" 갯수가 비슷해야 비슷한 수의 영화를 추천후보영화로 선정할 수 있습니다. <br><br>


In [14]:
ToyStory = 1 
Avengers3 = 122912
# 영화 목록

testMovieId = ToyStory
# 테스트 영화 번호

testMovie_all_tags = tm.get_all_tags_by_freq(testMovieId, tag_df)
# 테스트 영화의 모든 태그를 빈도순으로 뽑아냄
testMovieTF_tags = tm.get_TF_tags(testMovie_all_tags)
# 가중치가 10퍼센트 이상인 TF 태그 목록을 생성함 <-- 테스트 영화의 대표태그들을 의미함
testMovie_model_tags = tm.get_recommend_tags(testMovie_all_tags, model)
# 가중치가 10처센트 이상 + w2v모델 유사태그을 합친 태그 목록을 생성함 <-- 추천해줄만한 영화를 걸러내기 위해 사용됨
testMovie_model_tags


C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['disney animated feature',
 'want',
 'pixar',
 'dreamworks',
 'animation',
 'animated',
 'dragons',
 'disney',
 'anime',
 'computer animation']

In [15]:
recommend_movies_array = mm.get_recommend_movie_list(tag_df, rating_df, testMovie_model_tags)
# 추천영화목록을 만듬 <-- 평균평점 3점이상이고, 테스트영화의 모델 태그를 하나라도 보유한 영화를 뜻함.
recommend_movies_with_tags = mm.get_recommend_movie_with_all_tags(tag_df, recommend_movies_array)
# 추천영화목록의 영화들과 모든 태그를 보유한 데이터프레임을 생성함
recommend_TF = tm.get_TF_tags_for_many(recommend_movies_with_tags)
# 추천영화들의 가중치 10퍼센트 이상의 태그 목록을 생성함
result_df = rm.recommend(recommend_TF, testMovieTF_tags, model)
# 추천영화들의 태그와 테스트영화의 태그를 n x n 으로 매칭해서
# 유사도를 구해 그 순서로 정렬한 데이터 프레임을 뽑아냄

result_df = pd.merge(result_df, movie_df)
result_df = pd.merge(result_df, rating_df)
# 평점과 영화 이름을 포함시킴

result_df = rm.get_recommend_df(result_df, testMovieId)
# 추천지수를 포함한 마지막 결과값을 만들어냄

print(mm.id_to_name(testMovieId, movie_df), "\n")
print(testMovieTF_tags, "\n")
result_df.head(20)


C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy 

['pixar', 'animation', 'disney'] 



,movieId,recommendFactor,title,genres,tags,similarity,rating
0,120474,9.469373,Toy Story That Time Forgot (2014),Animation|Children,"[short, pixar, animation, steve purcell, toy s...",2.932368,3.229258
1,27738,8.749329,"Cathedral, The (Katedra) (2002)",Animation,"[sci-fi, animation, short films, 02/11, 01/11,...",2.524397,3.465909
2,162716,7.576805,The Old Mill (1937),Animation,"[short, short film, disney, animation, library...",2.096726,3.613636
5,164895,7.282428,You Are Umasou (2010),Adventure|Animation|Children,"[anime, japanese, heartwarming, rare, comedy, ...",1.867289,3.900000
3,2355,7.089646,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,"[pixar, animation, disney, funny]",1.982717,3.575722
12,170125,6.563338,Little Witch Academia: The Enchanted Parade (2...,Action|Animation|Comedy|Fantasy,"[magic, anime, magical girl, cute, visually ap...",1.742479,3.766667
4,88267,6.549930,Winnie the Pooh (2011),Animation|Children|Comedy,"[disney, hand-drawn, talking animals, animatio...",1.944928,3.367698
8,160874,6.394330,"Life, Animated (2016)",Animation|Documentary,"[documentary, inspirational, family, disney, c...",1.812017,3.528846
11,5218,6.235334,Ice Age (2002),Adventure|Animation|Children|Comedy,"[animation, pixar, funny, animated]",1.754729,3.553445
19,57543,6.133065,Fallen Art (Sztuka spadania) (2004),Action|Animation|Comedy,"[short, animation, not available from netflix,...",1.633349,3.754902
